In [9]:
import os.path
import numpy
import cv2
import csv
from six.moves import cPickle as pickle

In [10]:
#The path for the dataset images and for the csv file with
# the label information.
dataset_npy_path = "dataset.npy"
label_npy_path = "label.npy"
dataset_image_path = "C:/Users/sanke_000/HeadPoseEstimation/My Version/HeadPoseImageDatabase/output from prima parser/output"
label_csv_path = "C:/Users/sanke_000/HeadPoseEstimation/My Version/HeadPoseImageDatabase/output from prima parser/prima_label.csv"


In [11]:
#If the dataset numpy file do not exist it create it
if(os.path.isfile(label_npy_path)==False or os.path.isfile(dataset_npy_path)==False):

    #Saving the file names in a list
    image_list = list()
    with open(label_csv_path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            image_list.append(row[0])

    #Loading the label
    label = numpy.genfromtxt(label_csv_path, delimiter=',', skip_header=1, usecols=(3,4), dtype=numpy.float32)
    label_row, label_col = label.shape
    print(label.shape)
    print(label[0][:])

    dataset_row = label_row
    dataset_col = 64 * 64 #the size of the image
    dataset = numpy.zeros((dataset_row, dataset_col), dtype=numpy.int8)

    row_counter = 0
    for image_name in image_list:
        if image_name !='path':
            image_path = image_name
            image = cv2.imread(image_path, 0) #load in greyscale
            dataset[row_counter] = image.reshape(1, -1)
            row_counter += 1 

    #Saving the numpy array to files
    numpy.save("label", label)
    numpy.save("dataset", dataset)


(2790, 2)
[-90.   0.]


In [12]:
print("Loading the numpy file with the dataset and the label")

#Load the dataset and labels
#The dataset contains integers
#in the range -127/+127
label = numpy.load("./label.npy")
dataset_int = numpy.load("./dataset.npy")


print("Rescaling the image values from range -127/+127 to range -1/+1")

#Applying min-max scaling to the dataset
#After this step the data will be in the
# range -1/+1
dataset = dataset_int.astype(dtype=numpy.float32)
dataset /= 127 #dividing by the max value

# #Roll
# max_roll = numpy.amax(label[:,0])
# min_roll = numpy.amin(label[:,0])
# std_roll = numpy.std(label[:,0])
# mean_roll = numpy.mean(label[:,0])
# print("\n")
# print("===  ROLL  === \n" +  "mean: " + str(mean_roll) + "\n" + "std: " + str(std_roll) + "\n" + "max: " + str(max_roll) + "\n" + "min: " + str(min_roll) + "\n")

#Pitch
max_pitch = numpy.amax(label[:,0])
min_pitch = numpy.amin(label[:,0])
std_pitch = numpy.std(label[:,0])
mean_pitch = numpy.mean(label[:,0])
print("=== PITCH === \n" +  "mean: " + str(mean_pitch) + "\n" + "std: " + str(std_pitch) + "\n" + "max: " + str(max_pitch) + "\n" + "min: " + str(min_pitch) + "\n")

#Yaw
max_yaw = numpy.amax(label[:,1])
min_yaw = numpy.amin(label[:,1])
std_yaw = numpy.std(label[:,1])
mean_yaw = numpy.mean(label[:,1])
print("===  YAW  === \n" +  "mean: " + str(mean_yaw) + "\n" + "std: " + str(std_yaw) + "\n" + "max: " + str(max_yaw) + "\n" + "min: " + str(min_yaw) + "\n")

print("Rescaling the label values from radians to range -1/+1")


Loading the numpy file with the dataset and the label
Rescaling the image values from range -127/+127 to range -1/+1
=== PITCH === 
mean: 0.0
std: 38.667316
max: 90.0
min: -90.0

===  YAW  === 
mean: 0.0
std: 55.51809
max: 90.0
min: -90.0

Rescaling the label values from radians to range -1/+1


In [13]:
#Min-Max scaling of the label
# #Roll
# label[:,0] /= (numpy.pi/2) #pi/2 radians correspond to 90 degree
#Pitch
label[:,0] /= (numpy.pi/2) #pi/2 radians are 90 degree
#Yaw
label[:,1] /= (numpy.pi/2) #pi/2 radians correspond to 90 degree

# #Roll
# max_roll = numpy.amax(label[:,0])
# min_roll = numpy.amin(label[:,0])
# std_roll = numpy.std(label[:,0])
# mean_roll = numpy.mean(label[:,0])
# print("\n")
# print("===  ROLL Scaled  === \n" +  "mean: " + str(mean_roll) + "\n" + "std: " + str(std_roll) + "\n" + "max: " + str(max_roll) + "\n" + "min: " + str(min_roll) + "\n")

#Pitch
max_pitch = numpy.amax(label[:,0])
min_pitch = numpy.amin(label[:,0])
std_pitch = numpy.std(label[:,0])
mean_pitch = numpy.mean(label[:,0])
print("=== PITCH Scaled === \n" +  "mean: " + str(mean_pitch) + "\n" + "std: " + str(std_pitch) + "\n" + "max: " + str(max_pitch) + "\n" + "min: " + str(min_pitch) + "\n")

#Yaw
max_yaw = numpy.amax(label[:,1])
min_yaw = numpy.amin(label[:,1])
std_yaw = numpy.std(label[:,1])
mean_yaw = numpy.mean(label[:,1])
print("===  YAW Scaled  === \n" +  "mean: " + str(mean_yaw) + "\n" + "std: " + str(std_yaw) + "\n" + "max: " + str(max_yaw) + "\n" + "min: " + str(min_yaw) + "\n")


=== PITCH Scaled === 
mean: -8.75056e-08
std: 24.616377
max: 57.295776
min: -57.295776

===  YAW Scaled  === 
mean: -4.101825e-08
std: 35.343914
max: 57.295776
min: -57.295776



In [14]:
#Temporary append the label to the dataset to shuffle the data
data = numpy.append(dataset, label, axis=1)
#Shuffle the row to randomize the data
numpy.random.shuffle(data)

#Separating the label from the dataset
dataset = data[:,0:4096]
label = data[:,4096:4098]


In [17]:
#Creating the trainin and test datasets
# #Training dataset is 60% of the total
# row, col = dataset.shape
# cut_80 = int(row * 0.8) #take 80% of the total
# cut_10 = (row - cut_80)/2 #split the remaining 20%
training_dataset = numpy.copy(dataset[0:2604,:]) 
training_label = numpy.copy(label[0:2604,:]) 
# #Validation dataset is 10% of the total
# validation_dataset = numpy.copy(dataset[cut_80:cut_80+cut_10,:])
# validation_label = numpy.copy(label[cut_80:cut_80+cut_10,:])
# #Test dataset is 10% of the total
test_dataset = numpy.copy(dataset[2604:2791,:])
test_label = numpy.copy(label[2604:2791,:])


In [18]:
#saving the sanitized dataset in a pickle file
pickle_file = 'aflw_dataset.pickle'
print("Saving the dataset in: " + pickle_file)
print("... ")
try:
    f = open(pickle_file, 'wb')
    save = {
        'training_dataset': training_dataset,
        'training_label': training_label,       
        'test_dataset': test_dataset,
        'test_label': test_label        
        }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

print("The dataset has been saved and it is ready for the training! \n")


Saving the dataset in: aflw_dataset.pickle
... 
The dataset has been saved and it is ready for the training! 

